In [9]:
!pip3 install --trusted-host pypi.python.org --trusted-host files.pythonhosted.org --trusted-host pypi.org transformers

  Using cached transformers-4.27.4-py3-none-any.whl (6.8 MB)
  Using cached PyYAML-6.0-cp38-cp38-win_amd64.whl (155 kB)
  Using cached huggingface_hub-0.13.3-py3-none-any.whl (199 kB)
  Using cached tokenizers-0.13.2-cp38-cp38-win_amd64.whl (3.3 MB)


In [29]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
import zipfile
import os
# 불필요한 에러를 보여주지 않게 하는 os 설정
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from tensorflow.keras.utils import to_categorical
from tensorflow.python.client import device_lib
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout, Conv1D, GlobalMaxPooling1D,Input, Reshape, GlobalAveragePooling1D, GlobalMaxPooling1D, Concatenate, LayerNormalization, MultiHeadAttention, TimeDistributed, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
import re
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
import sys
from transformers import BertTokenizerFast, TFBertModel, TFBertForSequenceClassification, BertConfig
from transformers import pipeline

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yhunkim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yhunkim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
bert = "bert-base-uncased"
tokenizer = BertTokenizerFast.from_pretrained(bert)
model = TFBertForSequenceClassification.from_pretrained(bert, num_labels=8)
model

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 배열 전체 표시
np.set_printoptions(threshold=sys.maxsize)
len(model_w2v.wv)

In [65]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')


In [5]:
# print(sys.version)
print(device_lib.list_local_devices())

# GPU 사용 가능한지 확인
print("Num GPUs Available: ", tf.config.experimental.list_physical_devices('GPU'))

# Cuda 및 GPU 동작 확인
print(tf.test.is_built_with_cuda())
print(tf.test.is_built_with_gpu_support())
print(tf.test.gpu_device_name())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2094470795125646661
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4169138176
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3012637775583558840
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1660 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]
Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
True
True
/device:GPU:0


In [4]:
def preprocess_text(text):
    # 소문자로 변환
    text = text.lower()
    # 특수문자 제거
    text = re.sub(r"[^a-zA-Z]", " ", text)
    # 불용어 제거
    words = text.split()
    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word not in stop_words]
    # 어간(Stemming), 표제어(Lemmatization) 추출
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    # 텍스트로 다시 변환
    text = " ".join(words)
    return text

In [5]:
# model_w2v = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True, limit=500000)

with zipfile.ZipFile("open.zip", "r") as zip_f:
    
    print(zip_f.namelist())
    file_list = zip_f.namelist()
    df_list = []
    for f in file_list:
        
        
        with zip_f.open(f) as file:
            df = pd.read_csv(file)
            df_list.append(df)

    submission_df = df_list[0]
    train_df = df_list[2]
    test_df = df_list[1]

['sample_submission.csv', 'test.csv', 'train.csv']


In [6]:
# preprocessing
train_df['text'] = train_df['text'].apply(preprocess_text)
test_df['text'] = test_df['text'].apply(preprocess_text)

In [7]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(train_df['label'])
num_labels = len(label_encoder.classes_)

train_df['label'] = np.asarray(label_encoder.transform(train_df['label']), dtype=np.int32)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47399 entries, 0 to 47398
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      47399 non-null  object
 1   text    47399 non-null  object
 2   label   47399 non-null  int32 
dtypes: int32(1), object(2)
memory usage: 925.9+ KB


In [72]:
train_df[:20000]

,id,text,label
0,TRAIN_00000,israel parliament start winter session jerusal...,3
1,TRAIN_00001,two third business owner say prepared outbreak...,2
2,TRAIN_00002,story highlightsred bull team principal christ...,1
3,TRAIN_00003,final respect paid arafat palestinian pay last...,3
4,TRAIN_00004,steelers winning old fashioned way pound pitts...,1
...,...,...,...
19995,TRAIN_19995,toronto ottawa canadian technology firm shopif...,2
19996,TRAIN_19996,eu court annuls eu mln fine german bank update...,3
19997,TRAIN_19997,safran sa plan hire people world wide year fra...,2
19998,TRAIN_19998,central maine healthcare lewiston maine pandem...,0


In [73]:
train_texts = train_df['text'][:20000].to_list()
train_labels = train_df['label'][:20000].to_list()

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=42)

In [74]:
# Tokenizing
train_encodings = tokenizer.batch_encode_plus(train_texts, truncation=True, padding=True, max_length=64)
val_encodings = tokenizer.batch_encode_plus(val_texts, truncation=True, padding=True, max_length=64)

In [56]:
# trainset-set
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

# validation-set
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [70]:
# # 분류를 위한 추가 레이어 생성
# dropout = tf.keras.layers.Dropout(0.3)
# output = tf.keras.layers.Dense(8, activation='softmax')

# # 입력층 생성
# input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
# input_attention_masks = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_masks')

# # BERT 입력
# bert_output = bert({'input_ids': input_ids, 'attention_mask': input_attention_masks})[1]

# # 추가 레이어 적용
# dropout_output = dropout(bert_output)
# outputs = output(dropout_output)

# # 전체 모델 생성
# model = tf.keras.Model(inputs=[input_ids, input_attention_masks], outputs=outputs)

# 모델 컴파일
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
with tf.device('/GPU:0'):
    # model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    callback_earlystop = EarlyStopping(
        monitor="val_accuracy", 
        min_delta=0.001, # the threshold that triggers the termination (acc should at least improve 0.001)
        patience=2)

    model.fit(
        train_dataset.batch(64), epochs=10, batch_size=64,
        validation_data=val_dataset.batch(64),
        callbacks = [callback_earlystop]
    )

In [33]:
train_sen = [sentence.split() for sentence in train_df['text']]
test_sen = [sentence.split() for sentence in test_df['text']]


In [76]:
Vector_size = 200
model_w2v = Word2Vec(train_sen, vector_size=Vector_size, window=5, min_count=5, workers=8, sg=1)

In [116]:
# Split data into training and testing sets

X = np.array([np.mean([model_w2v.wv[word] for word in sentence if word in model_w2v.wv.key_to_index.keys()], axis=0) for sentence in train_sen])
test = np.array([np.mean([model_w2v.wv[word] for word in sentence if word in model_w2v.wv.key_to_index.keys()], axis=0) for sentence in test_sen])

In [131]:
y = train_df['label'].values
X_train_emb, X_val_emb, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBM 데이터셋 생성
train_data = lgb.Dataset(X_train_emb, label=y_train)
val_data = lgb.Dataset(X_val_emb, label=y_val)

In [150]:
# LGBMClassifier 모델 정의
lgbm = LGBMClassifier(random_state=42, device='gpu')

# 그리드 서치에서 탐색할 하이퍼파라미터 후보군
param_grid = {
    'num_leaves': [15, 31, 63],
    'max_depth': [3, 4, 5, 6, -1],
    'learning_rate': [0.01, 0.05, 0.1],
    'min_child_samples': [10, 20, 30],
    'bagging_fraction': [0.5, 0.7, 0.9, 1.0]
}

# 그리드 서치 수행
grid_search = GridSearchCV(lgbm, param_grid=param_grid, cv=5, scoring='f1_macro', n_jobs=-1)

with tf.device('/GPU:0'):
# LightGBM 모델 학습
    # lgb_model = lgb.train(params, train_data, valid_sets=[train_data, val_data], num_boost_round=1000, early_stopping_rounds=50, verbose_eval=50)
    grid_search.fit(X_train_emb, y_train)

[LightGBM] [Warning] bagging_fraction is set=0.5, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5


In [153]:
# 최적 하이퍼파라미터 출력
print('Best hyperparameters:', grid_search.best_params_)

# 최적 모델 저장
best_model = grid_search.best_estimator_
y_pred = best_model.predict(test)
np.unique(y_pred)

Best hyperparameters: {'bagging_fraction': 0.5, 'learning_rate': 0.1, 'max_depth': 4, 'min_child_samples': 20, 'num_leaves': 63}


array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int64)

In [154]:

submission_df = pd.read_csv('submission.csv')
submission_df['label'] = y_pred
submission_df.to_csv('wor2vec_lightgbm_CV.csv', index=False)

In [78]:
num_classes = len(np.unique(train_df['label']))
y = to_categorical(train_df['label'], num_classes)
X.shape

(47399, 200)

In [90]:
max_len = max([len(sentence) for sentence in X])
max_len

200

In [79]:
# Pad sequences to have equal length for LSTM 
### word2vec 초기 설정시 vector_size 
# max_len = max([len(sentence) for sentence in X])
# test_len = max([len(sentence) for sentence in test])
X = pad_sequences(X, maxlen=Vector_size, padding='post')
X_test = pad_sequences(test, maxlen=Vector_size, padding='post')

In [80]:
# 학습 및 검증 데이터셋 분할
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [89]:
len(model_w2v.wv.key_to_index)

100

In [110]:
# LSTM 모델 정의
# model = Sequential()
# model.add(Embedding(input_dim=len(model_w2v.wv.key_to_index), output_dim=100, input_length=max_len))
# model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(units=num_classes, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model = tf.keras.models.Sequential([
#   tf.keras.layers.Embedding(input_dim=len(model_w2v.wv.key_to_index), output_dim=Vector_size, input_length=max_len),
#   tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
#   tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
#   tf.keras.layers.Dense(64, activation='relu'),
#   tf.keras.layers.Dropout(0.5),
#   tf.keras.layers.Dense(num_classes, activation='softmax')
# ])
# 모델 하이퍼파라미터 설정


DENSE_SIZE = 256
NUM_HEADS = 8
DROPOUT_RATE = 0.5
LEARNING_RATE = 0.001
EPOCHS = 50
BATCH_SIZE = 64

# CNN 모델 구축
# model = Sequential()
# model.add(Embedding(input_dim=len(model_w2v.wv.key_to_index), output_dim=Vector_size, input_length=max_len))
# model.add(Conv1D(FILTERS, KERNEL_SIZE, activation='relu'))
# model.add(GlobalMaxPooling1D())
# model.add(Dense(HIDDEN_SIZE, activation='relu'))
# model.add(Dropout(DROPOUT_RATE))
# model.add(Dense(num_classes, activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 하이퍼파라미터 설정
MAX_SEQ_LEN = 200
EMBEDDING_DIM = 300
NUM_HEADS = 8
DROPOUT_RATE = 0.1
FFN_UNITS = 512

# 입력 레이어
inputs = Input(shape=(MAX_SEQ_LEN,), dtype=tf.int32)

# 임베딩 레이어
embedding_layer = Embedding(input_dim=len(model_w2v.wv.key_to_index)+ 1, output_dim=EMBEDDING_DIM, input_length=MAX_SEQ_LEN)(inputs)
dropout_layer = Dropout(DROPOUT_RATE)(embedding_layer)

# 위치 임베딩
position_embedding = tf.keras.layers.Embedding(input_dim=MAX_SEQ_LEN, output_dim=EMBEDDING_DIM)(tf.range(start=0, limit=MAX_SEQ_LEN, delta=1))
position_embedding = dropout_layer + position_embedding

# 멀티헤드 어텐션
attention_layer = MultiHeadAttention(num_heads=NUM_HEADS, key_dim=EMBEDDING_DIM, dropout=DROPOUT_RATE)(position_embedding, position_embedding)

# 어텐션과 임베딩을 연결
addition_layer = tf.keras.layers.Add()([attention_layer, position_embedding])
normalization_layer_1 = LayerNormalization(epsilon=1e-6)(addition_layer)

# FeedForward 레이어
feedforward_layer = TimeDistributed(Dense(units=FFN_UNITS, activation='relu'))(normalization_layer_1)
feedforward_layer = Dropout(DROPOUT_RATE)(feedforward_layer)
feedforward_layer = TimeDistributed(Dense(units=EMBEDDING_DIM))(feedforward_layer)

# 어텐션과 FeedForward 레이어를 연결
addition_layer = tf.keras.layers.Add()([feedforward_layer, normalization_layer_1])
normalization_layer_2 = LayerNormalization(epsilon=1e-6)(addition_layer)

# Global Max Pooling과 Global Average Pooling을 연결
pooling_layer = Concatenate()([GlobalMaxPooling1D()(normalization_layer_2), GlobalAveragePooling1D()(normalization_layer_2)])
dropout_layer = Dropout(DROPOUT_RATE)(pooling_layer)

# 출력 레이어
outputs = Dense(units=8, activation='softmax')(dropout_layer)

# 모델 생성
model = Model(inputs=inputs, outputs=outputs)

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4), metrics=['accuracy'])

c:\Users\yhunkim\Anaconda3\envs\chatgpt\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [65]:
# # GPU 사용 가능 여부 확인
# gpus = tf.config.list_physical_devices('GPU')
# if gpus:
#   try:
#     # GPU 사용 설정
#     for gpu in gpus:
      
#       print(gpu)
#       tf.config.experimental.set_memory_growth(gpu, True)
#     tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
#   except RuntimeError as e:
#     print(e)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Physical devices cannot be modified after being initialized


In [113]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True, save_weights_only=False, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=3, mode='min', verbose=1)

In [ ]:
# 모델 학습
with tf.device('/GPU:0'):
    model.fit(X_train, 
            y_train, 
            batch_size=64, 
            epochs=100, validation_data=(X_val, y_val),
            use_multiprocessing=True, workers=8,
            verbose=1)

In [72]:
model_name = "Word2vec" + f"_{100}"
model.save(f"{model_name}.h5")

In [138]:
y_test_pred = model.predict(test)
y_test_pred

2127/2605 [=======================>......] - ETA: 13s

KeyboardInterrupt: 

In [75]:
np.unique(y_test_pred.argmax(axis=1))

array([0], dtype=int64)